
# ViEWS 3 ensembles: future predictions
ViEWS monthly updates, cm level
Fatalities002 version

This notebook produces future predictions for a set of models defined in the list of dictionaries ModelList and the weights stored as iweights_df.csv. Both of these are produced by the notebook fatal_cm_compute_ensemble in this repository. 

The notebook draws on the following .py script files in this repository:

Ensembling.py

FetchData.py

ViewsEstimators.py

It also requires the list of models included in the ensemble, in the following file:

ModelDefinitions.py

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *
#import views_mapper2
#from views_mapper2.mapper2 import Mapper2
#from views_mapper2 import color
#from views_mapper2.label_writer import vid2date
#from views_mapper2.dictionary_writer import standard_scale

# Mapper
import geopandas as gpd

import sqlalchemy as sa
#from ingester3.config import source_db_path

# Other packages
import pickle as pkl

#Parallelization
from joblib import Parallel, delayed, cpu_count
from functools import partial

from pathlib import Path

# Packages from this repository, Tools folder
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
sys.path.append('../SystemUpdates')
import os
from pathlib import Path

from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated

from FetchData import FetchData, RetrieveFromList, ReturnQsList, get_df_from_datasets_by_name
from ViewsEstimators import *

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
# Common parameters:

dev_id = 'Fatalities002'
run_id = dev_id 
EndOfHistory = 530
prod_id = '2024_2_t01'
RunGeneticAlgo = False
level = 'cm'
WriteToOverleaf = False
get_future = False

username = os.getlogin()

steps = [*range(1, 36+1, 1)] # Which steps to train and predict for

#steps = [1,2,3,4,5,6,7,8,9,10,11,12,15,18,21,24] # Which steps to train and predict for
#fi_steps = [1,3,6,12,36] # Which steps to present feature importances for
#steps = [1,12,24,36]
fi_steps = [1,3,6,12,36]
#steps = [1,6,36]
#fi_steps = [1,6,36]

# Specifying partitions

calib_partitioner_dict = {"train":(121,408),"predict":(409,456)}
test_partitioner_dict = {"train":(121,456),"predict":(457,504)}
future_partitioner_dict = {"train":(121,504),"predict":(505,516)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

# Specifying paths - note these have to be set to conform to individual setups!

Mydropbox = f'/Users/{username}/ViEWS Dropbox/ViEWS/'
localgitpath = f'/Users/{username}/VIEWS3/'
notebookpath = os.getcwd()
markovpath = str(Path(notebookpath).parent.absolute())+'/Tools/markov/'

if WriteToOverleaf:
    if EndOfHistory==508:
        overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/ViEWS_Presentations_2021/Figures/Forecasts/Apr2022/'
    if EndOfHistory==509:
        overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/ViEWS_Presentations_2021/Figures/Forecasts/Apr2022/'
    
    print('Overleaf path set to',overleafpath)

print('Dropbox path set to:',Mydropbox)
print('Local GIT Path:', localgitpath)
print('Markov code path set to:',markovpath)


# Retrieve models and predictions

In [ ]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels(level)
    
i = 0
for model in ModelList:
    print(i, model['modelname'], model['data_train'])
    i = i + 1

# Retrieve and calibrate predictions and data

In [ ]:
### Running and saving David's models
## Import subprocess to run Rscript
#import subprocess

## Fetch and save data (can perhaps be simplified?)
#qs = Queryset('hh_20_features','country_month')
#qs.fetch().to_parquet(markovpath + 'tmp.parquet')

## Set commands and arguments. R-scripts located in 'Markov'-folder
#command ='Rscript'
##path2script ='../Tools/markov/omm_ranger_hh20_fcdo_py.R'
#path2script = markovpath + 'omm_ranger_hh20_fcdo_py.R'

#cmd = [command, path2script]
#data_path = markovpath + 'tmp.parquet'
#save_path = Mydropbox + 'Projects/PredictingFatalities/Predictions/cm/preds/'
#args = [str(EndOfHistory),data_path,save_path,]

## Run subprocess. Saves the predictions as csv-files to the save_path location with prefix vmm_[estimator]_hh20_[EndOfHistory]
#subprocess.call(cmd+args)


In [ ]:
## Retrieve David's models from dropbox and store in prediction storage
#path = Mydropbox + 'Projects/PredictingFatalities/Predictions/cm/preds/'

#DRList = [
#    {
#        'modelname': 'fat_hh20_Markov_glm',
#        'filename': path + 'vmm_glm_hh20_' + str(EndOfHistory) + '.csv'
#    },
    
#    {
#        'modelname': 'fat_hh20_Markov_rf',
#        'filename': path + 'vmm_rf_hh20_' + str(EndOfHistory) + '.csv'
#    }
#]
    
#for model in DRList:
#    df_future = pd.read_csv(model['filename'],index_col=['month_id','country_id'])
#    df_future['ln_ged_sb_dep'] = np.nan # Empty dependent variable column for consistency/required by prediction storage function
#    stored_modelname = level + '_' + model['modelname'] + '_f' + str(EndOfHistory)
#    df_future.forecasts.set_run(dev_id)
#    df_future.forecasts.to_store(name=stored_modelname, overwrite=True)    

In [ ]:
# Retrieving the predictions for calibration and test partitions
# The ModelList contains the predictions organized by model

ModelList = RetrieveStoredPredictions(ModelList, steps, EndOfHistory, dev_id, level, get_future)

ModelList = CalibratePredictions(ModelList, EndOfHistory, steps)

In [ ]:
# Run querysets and postprocessing (e.g. PCA) to obtain data for future prediction
qslist = ReturnQsList(level)
from FetchData import fetch_cm_data_from_model_def

Datasets=fetch_cm_data_from_model_def(qslist,EndOfHistory)

In [ ]:
# EndOfHistory can be reset here to facilitate rerunning several months without rereading input data
# Remove '#' and reset
#EndOfHistory = 506

In [ ]:
for ds in Datasets:
    df = ds['df']
    print(ds['Name'],df.isna().sum())
    ds['df']=df.fillna(0)

In [ ]:
for ds in Datasets:
    df = ds['df']
    print(ds['Name'],df.isna().sum())

In [ ]:
import sklearn

In [ ]:
print(sklearn.__version__)

In [ ]:
from views_runs import Storage, StepshiftedModels
from views_partitioning.data_partitioner import DataPartitioner
from viewser import Queryset, Column
from views_runs import operations
from views_runs.run_result import RunResult
from new_markov import markov
from pygam import LogisticGAM, LinearGAM, s, te

RewritePredictions = True # Set this to True to rewrite predictions even if they exist

force_retrain = True

def RetrainAndPredict(modelname):

    modelstore = storage.Storage()
    # Predictions for true future
    ct = datetime.now()
    print('Future', ct)
    modelstore = storage.Storage()
    model['RunResult_future']  = RunResult.retrain_or_retrieve(
            retrain            = force_retrain,
            store              = modelstore,
            partitioner        = DataPartitioner({"test":future_partitioner_dict}),
            stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
            dataset            = RetrieveFromList(Datasets,model['data_train']),
            queryset_name      = model['queryset'],
            partition_name     = "test",
            timespan_name      = "train",
            storage_name       = model['modelname'] + '_future',
            author_name        = "HH",
    )       
    predictions_future = model['RunResult_future'].run.future_point_predict(EndOfHistory,model['RunResult_future'].data)
    return predictions_future



i = 0
print('Computing predictions, production run ' + prod_id + ', development run ' + dev_id)
for model in ModelList[:]:

    # Loop that checks whether (1) this a model trained outside the main system, 
    # (2) retrieves the prediction if it exists in prediction storage,
    # (3) if not checks whether the trained model exists, retrains if not, 
    # Then calibrates the predictions and stores them if they have not been stored before for this run.
    # To do: set the data_preprocessing to the function in the model dictionary
    
    model['predstorename_ncal'] = level +  '_' + model['modelname'] + '_noncalibrated' + '_f' + str(EndOfHistory)
    model['predstorename_cal'] = level +  '_' + model['modelname'] + '_calibrated' + '_f' + str(EndOfHistory)

    
    if 'Markov' not in model['modelname']: # Only Markov models are currently exceptions
        print(i, model['modelname'])

        ct = datetime.now()
        print('Trying to retrieve non-calibrated predictions', ct)
        if RewritePredictions:
            model['future_df_noncalibrated'] = RetrainAndPredict(model['predstorename_ncal'])
        else:
            try:
                model['future_df_noncalibrated'] = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstorename_ncal'])
                print('Predictions for ', model['predstorename_ncal'], ', run', run_id, 'exist, retrieving from prediction storage')

            except KeyError:
                print(model['predstorename_ncal'], ', run', run_id, 'does not exist, predicting')
                model['future_df_noncalibrated'] = RetrainAndPredict(model['predstorename_ncal'])

        # Calibrating and storing   
        # Storing non-calibrated
        
        model['future_df_noncalibrated'].forecasts.set_run(run_id)
        model['future_df_noncalibrated'].forecasts.to_store(name=model['predstorename_ncal'], overwrite=True)   
        print('Calibrating')
        model['future_df_calibrated'] = model['future_df_noncalibrated'].copy()
        for step in steps:
            thismonth = EndOfHistory + step
            
            model['future_df_calibrated'].loc[thismonth,'step_combined'] = pd.DataFrame(model['calibration_gams'][step-1]['calibration_GAM'].predict(model['future_df_noncalibrated'].loc[thismonth])).values
         # Storing calibrated
        model['future_df_calibrated'].forecasts.set_run(run_id)
        model['future_df_calibrated'].forecasts.to_store(name=model['predstorename_cal'], overwrite=True)   
            
    else: # If one of David's Markov models
        print(i, model['modelname'])
            
        ct = datetime.now()
        print('Trying to retrieve non-calibrated predictions', ct)
        if RewritePredictions:
            model['future_df_noncalibrated'] = markov.compute_markov(test_partitioner_dict, EndOfHistory, model['depvar'], 'future', model['algorithm'])
        else:
            try:
                model['future_df_noncalibrated'] = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstorename_ncal'])
                print('Predictions for ', model['predstorename_ncal'], ', run', run_id, 'exist, retrieving from prediction storage')

            except KeyError:
                print(model['predstorename_ncal'], ', run', run_id, 'does not exist, predicting')
                model['future_df_noncalibrated'] = markov.compute_markov(test_partitioner_dict, EndOfHistory, model['depvar'], 'future', model['algorithm']) 
            
            
        model['future_df_noncalibrated'].forecasts.set_run(run_id)
        model['future_df_noncalibrated'].forecasts.to_store(name=model['predstorename_ncal'], overwrite=True) 
            
        model['future_df_calibrated'] = model['future_df_noncalibrated'].copy()
        
        model['future_df_calibrated']['step_combined']=pd.DataFrame(model['future_df_noncalibrated']['weighted_prediction'])
         # Storing calibrated
        
        model['future_df_calibrated'].forecasts.set_run(run_id)
        model['future_df_calibrated'].forecasts.to_store(name=model['predstorename_cal'], overwrite=True)   


    i = i + 1

print('All done')

In [ ]:
EnsembleList = [] # Separate list of dictionaries for ensembles!

Ensemble = {
    'modelname':            'genetic_ensemble',
    'algorithm':            [],
    'depvar':               'ln_ged_sb_dep',
    'data_train':           [],
    'Algorithm_text':       '',
    'calibration_gams':     [],
    'future_df_calibrated': [],
}
EnsembleList.append(Ensemble)


In [ ]:
# Collecting in one df, one column per model
ConstituentModels_df = pd.DataFrame(ModelList[0]['future_df_calibrated']['step_combined'])
ConstituentModels_df.columns = [ModelList[0]['modelname']]
for model in ModelList[:]:
    print(model['modelname'])
    ConstituentModels_df[model['modelname']] = pd.DataFrame(model['future_df_calibrated']['step_combined'])


In [ ]:
# Retrieve genetic algorithm results
i_weights_df = pd.read_csv('../Intermediates/GeneticWeights.csv')

# Retrieve ensemble predictions for test partition to create categorical predictions

In [ ]:
stored_modelname_test = level + '_' + 'ensemble_genetic' + '_test'

ensemble_test_df = pd.DataFrame.forecasts.read_store(stored_modelname_test, run=run_id)
ensemble_test_df.replace([np.inf, -np.inf], 0, inplace=True)  

ensemble_test_df

In [ ]:
# Generate dichotomous version of dependent variable
ensemble_test_df['ged_gte_25'] = ensemble_test_df['ln_ged_sb_dep'].apply(lambda x: 1 if x >= np.log1p(25) else 0)
# Generate multiclass version for uncertainty estimation
def ged_categorical(x):
    if x < np.log1p(0.5):
        return 0
    elif x < np.log1p(10):
        return 1
    elif x < np.log1p(100):
        return 2
    elif x < np.log1p(1000):
        return 3
    else :
        return 4

ensemble_test_df['ged_multi'] = ensemble_test_df['ln_ged_sb_dep'].apply(ged_categorical)

ensemble_test_df.describe()

In [ ]:
plt.scatter(ensemble_test_df['ln_ged_sb_dep'],ensemble_test_df['ged_multi'])

In [ ]:
# Train model to transform predictions from  fatalities to (1) dichotomous and (2) multiclass
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
dichotomous_classifiers = []
multi_classifiers = []
for step in steps:
    X = np.array(ensemble_test_df[f'step_pred_{step}'])
    X = X.reshape(-1,1)
    # Dichotomous
    y_dich = np.array(ensemble_test_df['ged_gte_25']).reshape(-1, 1)
    dich_clf = LogisticRegression(random_state=0).fit(X, y_dich)
    p_dich = dich_clf.predict_proba(X)
    ensemble_test_df[f'dich_step_{step}_logit'] = p_dich[:,1].ravel()
    # Calibrated
    calibrated_dich_clf = CalibratedClassifierCV(base_estimator=dich_clf, cv=3)
    calibrated_dich_clf.fit(X, y_dich)
    p_dich_cal = calibrated_dich_clf.predict_proba(X)
    dichotomous_classifiers.append(calibrated_dich_clf)
    ensemble_test_df[f'dich_cal_step_{step}_logit'] = p_dich_cal[:,1].ravel()
    # Multiclass
    y_multi = np.array(ensemble_test_df['ged_multi']).reshape(-1, 1)
    multi_clf = LogisticRegression(random_state=0).fit(X, y_multi)
    multi_classifiers.append(multi_clf)
    p_multi = multi_clf.predict_proba(X)
    for cls in [0,1,2,3,4]:
        ensemble_test_df[f'multi_{cls}_step_{step}_logit'] = p_multi[:,cls].ravel()

ensemble_test_df[['dich_step_3_logit','dich_cal_step_3_logit']].describe()

In [ ]:
plt.scatter(ensemble_test_df['dich_step_3_logit'],ensemble_test_df['dich_cal_step_3_logit'])

# Calculating and storing ensemble future predictions

In [ ]:
# Setting up a placeholder df for ensemble predictions
EnsembleList[0]['future_df_calibrated'] = ModelList[0]['future_df_calibrated'].copy() # Copy from baseline
EnsembleList[0]['future_df_dichotomous'] = ModelList[0]['future_df_calibrated'].copy() # Copy from baseline

df=EnsembleList[0]['future_df_calibrated'].fillna(0)
EnsembleList[0]['future_df_calibrated']=df
df=EnsembleList[0]['future_df_dichotomous'].fillna(0)
EnsembleList[0]['future_df_dichotomous']=df


ConstituentModels_df_w = ConstituentModels_df.copy().fillna(0)

for step in steps:
    month = EndOfHistory + step
    weightcol = 'step_pred_' + str(step)
    weights = np.array(pd.DataFrame(i_weights_df[weightcol]))
    EnsembleList[0]['future_df_calibrated'].loc[month] = ConstituentModels_df_w.loc[month].dot(weights).values
    x_d = np.array(EnsembleList[0]['future_df_calibrated'].loc[month]).reshape(-1,1)
    pred_step = dichotomous_classifiers[step-1].predict_proba(x_d)
    EnsembleList[0]['future_df_dichotomous']['step_combined'].loc[month] = pred_step[:,1]

In [ ]:
# Storing the ensemble future predictions
predstore_future = level +  '_' + EnsembleList[0]['modelname'] + '_f' + str(EndOfHistory)
EnsembleList[0]['future_df_calibrated'].forecasts.set_run(run_id)
EnsembleList[0]['future_df_calibrated'].forecasts.to_store(name=predstore_future, overwrite = True) 
predstore_future_dich = level +  '_' + EnsembleList[0]['modelname'] + '_dich_f' + str(EndOfHistory)
EnsembleList[0]['future_df_dichotomous'].forecasts.set_run(run_id)
EnsembleList[0]['future_df_dichotomous'].forecasts.to_store(name=predstore_future_dich, overwrite = True) 


In [ ]:
ViewsMetadata().with_name('genetic').fetch()

## Retrain the surrogate models

In [ ]:
Datasets[1]['df'].loc[544]

In [ ]:
from cm_surrogatemodels import TrainSurrogateModels
SurrogateModelSteps = [1,3,6,36]
SurrogateModelSteps = steps
EndOfHistory_test = test_partitioner_dict['train'][1] 
Plotpath = Mydropbox + 'Projects/PredictingFatalities/SurrogateModels/'

df_all_features = get_df_from_datasets_by_name(Datasets,'all_features')

# Datasets[3] is (currently) the dataframe with all features:
#print('Dataset with input features:', Datasets[3]['Name'])
       
SurrogateModelList = TrainSurrogateModels(data_df = df_all_features, 
                                          Ensemble_df = ensemble_test_df, 
                                          EndOfHistory = EndOfHistory_test, 
                                          SurrogateModelSteps = SurrogateModelSteps, 
                                          NumberOfMonths = 48,
                                          Plotpath = Plotpath)

In [ ]:
predictors_df = df_all_features.loc[EndOfHistory]

if predictors_df.isna().sum().sum()>0:
    print('Warning - nulls found in predictors',predictors_df.isna().sum().sum())
    predictors_df = predictors_df.fillna(0)    


EnsembleList[0]['future_df_surrogates'] = EnsembleList[0]['future_df_calibrated'].copy()
# Initialize dataframe to hold surrogate model predictions:
for item in SurrogateModelList:
    if item['Step'] == 1:
        colname = item['Modelname'][item['Modelname'].index(' ') + 1:] # Remove first word (which is a step number)
        EnsembleList[0]['future_df_surrogates'][colname] = np.nan  
# Compute predictions for each step
for step in steps:
    month = EndOfHistory + step
#    print('Step',step,'Month',month)
    for item in SurrogateModelList:
        colname = item['Modelname'][item['Modelname'].index(' ') + 1:] # Remove first word (which is a step number)
        if item['Step']==step:
#            print('colname:',colname,'Step:',item['Step'], item['Columns'])
            EnsembleList[0]['future_df_surrogates'][colname].loc[month] = item['GAM'].predict(predictors_df[item['Columns']])

# Storing the surrogate model future predictions
api_definition = []
for item in SurrogateModelList:
    if item['Step'] == 36:
        colname = item['Modelname'][item['Modelname'].index(' ') + 1:] # Remove first word (which is a step number)
        predstore_future = level +  '_surrogate_' + item['Shortname'] + '_f' + str(EndOfHistory)
        print('Storing surrogate model predictions for model',colname, 'as:',predstore_future)
        predictions_to_store = pd.DataFrame(EnsembleList[0]['future_df_surrogates'][colname])
        predictions_to_store.forecasts.set_run(run_id)
        predictions_to_store.forecasts.to_store(name=predstore_future, overwrite = True) 
        api_item = {
            'Dev_id': dev_id,
            'EndOfHistory': EndOfHistory,
            'Model': colname,
            'Prediction storage colname': predstore_future
        }
        api_definition.append(api_item)

api_definition

In [ ]:
import json
with open( '../Intermediates/api_definition.json', 'w') as api_file:
    json.dump(api_definition,api_file)

In [ ]:
for model in SurrogateModelList:
    if model['Step'] == 1:
        print(model['Modelname'])

# Documentation

### Document Surrogates

This section takes the defined Surrogate models, creates a df, converts the df to md, and saves the md file in the appropriate path on GitHub ('viewsforecasting/ModelDocumentation/Surrogates/cm/')

In [ ]:
from FetchData import SurrogateMetadata
modelpath = localgitpath + 'viewsforecasting/Documentation/ModelDocumentation/Surrogates/cm/'
metadata = SurrogateMetadata(SurrogateModelList)
metadata.to_markdown(path= modelpath+'SurrogateModels.Md')
metadata.surrogate_model_list

### Document Ensembles

This section takes the defined Ensemble models, creates a df, converts the df to md, and saves the md file in the appropriate path on GitHub ('viewsforecasting/ModelDocumentation/Ensembles/cm/')

In [ ]:
from ModelDefinitions import DefineEnsembleModels

EnsembleModelList = DefineEnsembleModels(level)
df3 = pd.DataFrame(EnsembleModelList, columns=['modelname','description','depvar','queryset', 'algorithm','long_description']) 
#This cell assigns the file save path and converts the df to markdown
modelpath = localgitpath + 'viewsforecasting/Documentation/ModelDocumentation/Ensembles/cm/'
path= modelpath+'EnsembleModels.Md'
df3.to_markdown(buf=path)

# Uncertainty of predictions

In [ ]:
# Train model to transform predictions from  fatalities to multiclass probabilities
from sklearn.linear_model import LogisticRegression
# Classes are: 
# 0: Less than 0.5
# 1: 0.5-10
# 2: 10-100
# 3: 100-1000
# 4: 1000 +

multi_classifiers = []
df_future = EnsembleList[0]['future_df_calibrated'].copy()
for cls in [0,1,2,3,4]:
    df_future[f'multi_{cls}_logit'] = np.nan

for step in steps:
    Month = EndOfHistory + step
    X = np.array(ensemble_test_df[f'step_pred_{step}'])
    X = X.reshape(-1,1)
    # Multiclass
    y_multi = np.array(ensemble_test_df['ged_multi']).reshape(-1, 1)
    multi_clf = LogisticRegression(random_state=0).fit(X, y_multi)
    multi_classifiers.append(multi_clf)
    X_future = np.array(df_future['step_combined'].loc[Month]).reshape(-1,1)
    p_multi = multi_clf.predict_proba(X_future)
    for cls in [0,1,2,3,4]:
        df_future[f'multi_{cls}_logit'].loc[Month] = p_multi[:,cls]

# Storing the multi predictions in prediction storage:
for cls in [0,1,2,3,4]:
    predstore_future_multi = level +  '_multi_' + str(cls) + '_f' + str(EndOfHistory)
    print('Storing multiclass model predictions as:',predstore_future_multi)
    colname = 'multi_' + str(cls) + '_logit'
    predictions_to_store = pd.DataFrame(df_future[colname])
    predictions_to_store.forecasts.set_run(run_id)
    predictions_to_store.forecasts.to_store(name=predstore_future_multi, overwrite = True) 


df_future.describe()

In [ ]:
df_future.head()

In [ ]:
# Some uncertainty calculations
#October 2022 (514)
CL = [
    ('Ethiopia',57,4.114,0.0025217435284640467,0.167910951747582,0.7099704830039664,0.11880956751085855,0.0007872542091291349),
    ('Kenya',237,2.202,0.27979924123523675,0.37093623451842744,0.3384779788333302,0.010780625441912769,5.919971092604168e-06),
    ('Nigeria',79,5.891,1.1834522019553202e-05,0.030050677139008334,0.5285925149395453,0.41358824669460437,0.027756726704822352),
    ('South Africa',163,0.103,0.9794051664145842,0.017598332401191557,0.0029811455728619585,1.5355064175926178e-05,5.471865662986281e-10),
    ('South Sudan',246,1.782,0.5171316993204126,0.2898656997683062,0.18882479925510778,0.004176475005701126,1.3266504722650525e-06),
    ('Sudan',245,1.971,0.40522067161826564,0.3345539054572092,0.2536133366482648,0.006609400618818177,2.6856574423520527e-06),
    ('Syria',220,4.818,0.0003291633453796389,0.09280289507829718,0.6904155754185439,0.21292026113053514,0.0035321050272440497),
    ('Tanzania',242,0.741,0.9214126374988004,0.06115410932346314,0.017278449601019125,0.0001547909157059216,1.2661011272757968e-08),
    ('Yemen',124,6.352,2.556591139139699e-06,0.016708050733656395,0.42553124292260786,0.4969226339388311,0.06083551581376548),
    ('Zimbabwe',158,0.050,0.9816070866339232,0.015813575350686226,0.0025667146500692674,1.2622945665883365e-05,4.196556261097782e-10),
]

for C in CL: 
    print(C[0],C[2],np.expm1(C[2]))
    print('< 0.5:',C[3])
    print('0.5-10:',C[4])
    print('10-100:',C[5])
    print('100-1000:',C[6])
    print('1000+:',C[7])
    print('****')
    

In [ ]:
df_future.to_csv('Categorical_probabilities.csv')